# Calculate UTCI adjustment for `tasmax` data from regional UKCP

## Data

In [1]:
import iris

In [5]:
def load_ukcp_region(var, region="London"):
    fname = f'data/UKCP/{var}_regional/{var}_rcp85_land-cpm_uk_region_*.nc'
    cubes = iris.load(fname, iris.Constraint(Region=region))
    iris.util.equalise_attributes(cubes)
    return cubes.concatenate()[0]

In [2]:
fname = 'data/UKCP/tasmax_regional/tasmax_rcp85_land-cpm_uk_region_*.nc'
cubes = iris.load(fname, iris.Constraint(Region="London"))
iris.util.equalise_attributes(cubes)
tasmax, = cubes.concatenate()
tasmax.summary(shorten=True)

'air_temperature / (degC)            (ensemble_member: 12; time: 21600)'

In [3]:
fname = 'data/UKCP/hurs_regional/hurs_rcp85_land-cpm_uk_region_*.nc'
cubes = iris.load(fname, iris.Constraint(Region="London"))
iris.util.equalise_attributes(cubes)
hurs, = cubes.concatenate()
hurs.summary(shorten=True)

'relative_humidity / (%)             (ensemble_member: 12; time: 21600)'

In [4]:
fname = 'data/UKCP/sfcWind_regional/sfcWind_rcp85_land-cpm_uk_region_*.nc'
cubes = iris.load(fname, iris.Constraint(Region="London"))
iris.util.equalise_attributes(cubes)
wind, = cubes.concatenate()
wind.summary(shorten=True)

'wind_speed / (m s-1)                (ensemble_member: 12; time: 21600)'

## UTCI calculation

In [8]:
import pymeteosalute
import numpy as np
utci_v = np.vectorize(pymeteosalute.utci)

In [10]:
%%time
utci_data = utci_v(tasmax.data, hurs.data, wind.data, tasmax.data)
utci_data

CPU times: user 17.6 s, sys: 1.36 s, total: 19 s
Wall time: 24.9 s


masked_array(
  data=[[-2.50212705, -5.38497801, -6.18383294, ...,  4.3681903 ,
          4.59805613,  3.85521946],
        [ 6.0322373 ,  2.22949893, -0.58125055, ...,  4.20642437,
          3.556917  ,  2.29229037],
        [-0.53812088,  0.91224009,  1.59225111, ...,  0.61840506,
          0.26822985, -5.37464   ],
        ...,
        [-5.38468308, -7.7236601 , -3.53396972, ...,  7.24410984,
          4.84212432,  4.32258359],
        [ 1.69346529, -3.36349398, -3.43035787, ..., -2.52208689,
          6.90383024,  8.65254836],
        [-0.32030747,  3.84509927, -3.5637206 , ...,  8.0429015 ,
          7.49478189, 10.15230661]],
  mask=False,
  fill_value=1.0000000200408773e+20)

In [24]:
utci_cube = tasmax.copy(data=utci_data)
utci_cube.rename("utci")
utci_cube.attributes['description'] = "Maximum air temperature (UTCI adjusted)"
utci_cube

Utci (degC),ensemble_member,time
Shape,12,21600
Dimension coordinates,,
ensemble_member,x,-
time,-,x
Auxiliary coordinates,,
ensemble_member_id,x,-
month_number,-,x
year,-,x
yyyymmdd,-,x
Scalar coordinates,,


## Save to file

In [31]:
%%time
iris.save(utci_cube, 'data/UKCP/utci_regional/utcimax_rcp85_land-cpm_uk_london_01-15_day_19801201-20801130.nc')

CPU times: user 3.73 s, sys: 76.6 ms, total: 3.81 s
Wall time: 4.08 s


## UTCI Categorical calculation

In [32]:
utci10_v = np.vectorize(pymeteosalute.utci_class_10)

In [33]:
%%time
utci10_data = utci10_v(tasmax.data, hurs.data, wind.data, tasmax.data)
utci10_data

CPU times: user 8.55 s, sys: 95.7 ms, total: 8.64 s
Wall time: 8.92 s


masked_array(
  data=[[4., 4., 4., ..., 5., 5., 5.],
        [5., 5., 4., ..., 5., 5., 5.],
        [4., 5., 5., ..., 5., 5., 4.],
        ...,
        [4., 4., 4., ..., 5., 5., 5.],
        [5., 4., 4., ..., 4., 5., 5.],
        [4., 5., 4., ..., 5., 5., 6.]],
  mask=False,
  fill_value=1.0000000200408773e+20)

In [38]:
utci10_cube = tasmax.copy(data=utci10_data)
utci10_cube.rename("utci class 10")
utci10_cube.units = "1"
utci10_cube.attributes['description'] = "UTCI Heat Stress Category (daily max temperature)"
utci10_cube

Utci Class 10 (1),ensemble_member,time
Shape,12,21600
Dimension coordinates,,
ensemble_member,x,-
time,-,x
Auxiliary coordinates,,
ensemble_member_id,x,-
month_number,-,x
year,-,x
yyyymmdd,-,x
Scalar coordinates,,


## Save to file

In [39]:
%%time
iris.save(utci10_cube, 'data/UKCP/utci_regional/utci10max_rcp85_land-cpm_uk_london_01-15_day_19801201-20801130.nc')

CPU times: user 4.22 s, sys: 92.3 ms, total: 4.31 s
Wall time: 4.69 s
